In [1]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)

In [13]:
class SetupEvent(Event):
    query: str


class StepTwoEvent(Event):
    query: str


class StatefulFlow(Workflow):
    @step
    async def start(
        self, ctx: Context, ev: StartEvent
    ) -> SetupEvent | StepTwoEvent:
        db = await ctx.get("some_database", default=0)
        if db is None or db == 0:
            print("Need to load data")
            return SetupEvent(query=ev.query)
        # do something with the query
        return StepTwoEvent(query=ev.query)
    
    @step
    async def step_two(self, ctx: Context, ev: StepTwoEvent) -> StopEvent:
        # do something with the data
        resutl = await ctx.get("some_database")
        print("Data is ", resutl)
        return StopEvent(result= resutl[1])

    @step
    async def setup(self, ctx: Context, ev: SetupEvent) -> StartEvent:
        # load data
        await ctx.set("some_database", [1, 2, 3])
        return StartEvent(query=ev.query)

In [14]:
w = StatefulFlow(timeout=10, verbose=True, disable_validation=True)
result = await w.run(query="Some query")
print(result)

Running step start
Need to load data
Step start produced event SetupEvent
Running step setup
Step setup produced event StartEvent
Running step start
Step start produced event StepTwoEvent
Running step step_two
Data is  [1, 2, 3]
Step step_two produced event StopEvent
2
